In [60]:
import numpy as np
import pandas as pd
import mediapipe as mp
import cv2
import os
import math
import re


In [61]:
mp_pose = mp.solutions.pose


In [62]:
landmarks_points = {
    "left_shoulder": 0, "right_shoulder": 1,
    "left_elbow": 2, "right_elbow": 3,
    "left_wrist": 4, "right_wrist": 5,
    "left_hip": 6, "right_hip": 7,
    "left_knee": 8, "right_knee": 9,
    "left_ankle": 10, "right_ankle": 11,
    "left_heel": 12, "right_heel": 13,
    "left_foot_index": 14, "right_foot_index": 15,
}


In [ ]:
col_names = {
    "class",
    step,
    armpit_left,
    armpit_right,elbow_left,elbow_right,hip_left,hip_right,knee_left,knee_right,ankle_left,ankle_right
}


In [63]:
# def angle(point1, point2, point3):
#     if(point1==(0,0) or point2==(0,0) or point3==(0,0)):
#         return 0
#     numerator = point2[1] * (point1[0] - point3[0]) + point1[1] * \
#                 (point3[0] - point2[0]) + point3[1] * (point2[0] - point1[0])
#     denominator = (point2[0] - point1[0]) * (point1[0] - point3[0]) + \
#                 (point2[1] - point1[1]) * (point1[1] - point3[1])
#     try:
#         ang = math.atan(numerator/denominator)
#         ang = ang * 180 / math.pi
#         if ang < 0:
#             ang = 180 + ang
#         return ang
#     except:
#         return 90.0

def angle(pt1, pt2, pt3):
    angle_calc = 0
    a = (pt1[0], pt1[1])
    b = (pt2[0], pt2[1])
    c = (pt3[0], pt3[1])
    ang = math.degrees(math.atan2(
        c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    angle_calc = ang + 360 if ang < 0 else ang
    angle_calc = 360-angle_calc if angle_calc > 215 else angle_calc
    return angle_calc


In [64]:
pose_list = []

main_dir = "./poses_dataset/Images"
pose_dir_list = os.listdir(main_dir)

for i in range(8, len(pose_dir_list)):
    if i in [0, 2, 6, 7, 9]:
        continue
    step_list = []
    wrong_right_dir_list = os.listdir(f"{main_dir}/{pose_dir_list[i]}")

    for j in range(0, len(wrong_right_dir_list) - 1):
        steps_dir_list = os.listdir(
            f"{main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}")

        for k in range(0, len(steps_dir_list) - 1):
            sl = []
            images_dir_list = os.listdir(
                f"{main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}/{steps_dir_list[k]}")
            if j == 1:
                reg = re.search(r"([0-9])", steps_dir_list[k])

            for l in range(0, len(images_dir_list)):
                pre_list = []
                with mp_pose.Pose(static_image_mode=True, enable_segmentation=True) as pose:
                    image = cv2.imread(
                        f"{main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}/{steps_dir_list[k]}/{images_dir_list[l]}")
                    result = pose.process(
                        cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

                    try:
                        predict = True
                        for landmarks in result.pose_landmarks.landmark:  # change to pose_world_landmarks if needed
                            pre_list.append(landmarks)
                    except AttributeError:
                        print(
                            f"No points {main_dir}/{pose_dir_list[i]}/{wrong_right_dir_list[j]}/{steps_dir_list[k]}/{images_dir_list[l]}")
                        predict = False

                if predict == True:
                    gen1116 = np.array([
                        [
                            pre_list[m].x,
                            pre_list[m].y
                        ] for m in range(11, 17)
                    ]).tolist()

                    gen2333 = np.array([
                        [
                            pre_list[m].x,
                            pre_list[m].y
                        ] for m in range(23, 33)
                    ]).tolist()

                    gen1116.extend(gen2333)

                    all_lm = gen1116.copy()

                    pre_list = []

                    armpit_left = angle(
                        all_lm[landmarks_points["left_elbow"]],
                        all_lm[landmarks_points["left_shoulder"]],
                        all_lm[landmarks_points["left_hip"]]
                    )
                    armpit_right = angle(
                        all_lm[landmarks_points["right_elbow"]],
                        all_lm[landmarks_points["right_shoulder"]],
                        all_lm[landmarks_points["right_hip"]]
                    )

                    elbow_left = angle(
                        all_lm[landmarks_points["left_shoulder"]],
                        all_lm[landmarks_points["left_elbow"]],
                        all_lm[landmarks_points["left_wrist"]]
                    )
                    elbow_right = angle(
                        all_lm[landmarks_points["right_shoulder"]],
                        all_lm[landmarks_points["right_elbow"]],
                        all_lm[landmarks_points["right_wrist"]]
                    )

                    hip_left = angle(
                        all_lm[landmarks_points["right_hip"]],
                        all_lm[landmarks_points["left_hip"]],
                        all_lm[landmarks_points["left_knee"]]
                    )
                    hip_right = angle(
                        all_lm[landmarks_points["left_hip"]],
                        all_lm[landmarks_points["right_hip"]],
                        all_lm[landmarks_points["right_knee"]]
                    )

                    knee_left = angle(
                        all_lm[landmarks_points["left_hip"]],
                        all_lm[landmarks_points["left_knee"]],
                        all_lm[landmarks_points["left_ankle"]]
                    )
                    knee_right = angle(
                        all_lm[landmarks_points["right_hip"]],
                        all_lm[landmarks_points["right_knee"]],
                        all_lm[landmarks_points["right_ankle"]]
                    )

                    ankle_left = angle(
                        all_lm[landmarks_points["left_knee"]],
                        all_lm[landmarks_points["left_ankle"]],
                        all_lm[landmarks_points["left_foot_index"]]
                    )
                    ankle_right = angle(
                        all_lm[landmarks_points["right_knee"]],
                        all_lm[landmarks_points["right_ankle"]],
                        all_lm[landmarks_points["right_foot_index"]]
                    )

                    pre_list = [
                        i,
                        j,
                        armpit_left,
                        armpit_right,
                        elbow_left,
                        elbow_right,
                        hip_left,
                        hip_right,
                        knee_left,
                        knee_right,
                        ankle_left,
                        ankle_right
                    ]

                    print(pre_list)

                    sl.append(pre_list)


[8, 0, 54.86742951334198, 56.299743991561, 208.208031523484, 154.1230945646258, 82.07044385976809, 60.94270433016125, 211.98619856635065, 204.38277161506971, 46.04435831146708, 19.866983851113616]
[8, 0, 56.931788911315536, 46.89142695962158, 210.30230033293932, 162.46763431384827, 40.69963829436807, 143.61649349374412, 200.33010528322592, 163.64417234592162, 133.03590721351068, 101.57154379875783]
[8, 0, 56.99919336719302, 33.571028348125196, 134.6953887448074, 167.72621301512362, 20.494451996829298, 164.6432273983534, 191.82389240307964, 176.9187705617884, 208.7526782759691, 128.8770710431291]
[8, 0, 46.45281286248867, 6.555484454451403, 123.47451424743065, 210.21583920060374, 13.17270392269586, 168.47756515846612, 182.57071457211816, 175.9437232687887, 207.14696418561056, 142.28187332529626]
[8, 0, 18.213870194920787, 2.923445583830714, 213.1361812022032, 134.523102562384, 200.57251733392104, 19.536662910085024, 182.9883039490293, 181.77937558332394, 201.6649683027318, 207.941231467

In [65]:
data = pd.DataFrame(sl)
print(data[2].median())
print(data[3].median())
print(data[4].median())
print(data[5].median())
print(data[6].median())
print(data[7].median())


44.69022616193224
40.15613844686902
186.62608716360285
137.82624445977265
83.07999714631055
100.64143400095254
